# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob



In [4]:

PRICE_DATA = os.getenv("PRICE_DATA")


In [5]:
parquet_files = glob(os.path.join(PRICE_DATA, "**","*.parquet"), recursive = True)

For each ticker and using Dask, do the following:

Add lags for variables Close and Adj_Close.

Add returns based on Close:

returns: (Close / Close_lag_1) - 1
Add the following range:

hi_lo_range: this is the day's High minus Low.
Assign the result to dd_feat.

In [6]:
# Write your code below.

In [7]:
dd_prices = dd.read_parquet(parquet_files)

dd_prices["Close_lag_1"] = dd_prices.groupby("ticker")["Close"].shift(1)
dd_prices["Adj_Close_lag_1"] = dd_prices.groupby("ticker")["Adj Close"].shift(1)

dd_prices["returns"] = dd_prices["Close"] / dd_prices["Close_lag_1"] - 1
dd_prices["hi_lo_range"] = dd_prices["High"] - dd_prices["Low"]

dd_feat = dd_prices

C:\Users\Yenko\AppData\Local\Temp\ipykernel_30284\4205224717.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .shift(1)
  After:  .shift(1, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(1, meta=('x', 'f8'))            for series result
  dd_prices["Close_lag_1"] = dd_prices.groupby("ticker")["Close"].shift(1)
C:\Users\Yenko\AppData\Local\Temp\ipykernel_30284\4205224717.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .shift(1)
  After:  .shift(1, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(1, meta=('x', 'f8'))            for series result
  dd_prices["Adj_Close_lag_1"] = dd_prices.groupby("ticker")["Adj Close"].shift(1)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [11]:
import pandas as pd

In [12]:
dd_feat = dd_prices

In [9]:
# Write your code below.
pdf = dd_feat.compute()



In [13]:
pdf["Date"] = pd.to_datetime(pdf["Date"], errors="coerce")
pdf = pdf.dropna(subset=["Date"])
pdf = pdf.sort_values(["ticker", "Date"])

# 10-day (10-row) moving average of returns
pdf["returns_ma_10"] = (
    pdf.groupby("ticker")["returns"]
       .rolling(10)
       .mean()
       .reset_index(level=0, drop=True)
)

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
Answer: no it was not required, however it helped to simplify the calculations
+ Would it have been better to do it in Dask? Why?
Answer: For larger datasets or production-style pipelines it is better to do this feature engineering using Dusk.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.